# iterator 를 만드는 메소드
- `.make_one_shot_iterator()`
- `.make_initializable_iterator()`
### `tf.data.Iterator`
https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/data/Iterator

# dataset 를 만드는 메소드
- `from_generator` 
- `from_sparse_tensor_slices` 
- `from_tensor_slices`
- `from_tensors`

# functional 메소드
- filter
- map
- flat_map
- reduce
- shard
- repeat
- shuffle
- batch
- window
- take
- skip
- prefetch
- interleave
- apply
- zip
- concatenate

# graph based vs. eager execution

**eager execution** 의 경우, dataset 의 iterator 생성 메소드를 통해서 얻은 iterator 를 바로 사용할 수 있음.

In [1]:
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.data import Dataset

dataset = Dataset.from_tensor_slices([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
dataset = dataset.flat_map(lambda x: Dataset.from_tensor_slices(x))
iterator = dataset.make_one_shot_iterator()

for item in iterator:
    print(item)

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


**eager execution** 이 아니면, `TypeError: 'Iterator' object is not iterable` 발생

In [1]:
import tensorflow as tf
# tf.enable_eager_execution()
from tensorflow.data import Dataset

dataset = Dataset.from_tensor_slices([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
dataset = dataset.flat_map(lambda x: Dataset.from_tensor_slices(x))
iterator = dataset.make_one_shot_iterator()

for item in iterator:
    print(item)

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


TypeError: 'Iterator' object is not iterable

session 을 만들어서 `get_next` 로 얻은 element 를 참조해서 사용한다.

In [5]:
import tensorflow as tf
# tf.enable_eager_execution()
from tensorflow.data import Dataset

dataset = Dataset.from_tensor_slices([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
dataset = dataset.flat_map(lambda x: Dataset.from_tensor_slices(x))
iterator = dataset.make_one_shot_iterator()

el = iterator.get_next()

with tf.Session() as sess:
    for i in range(9):
        print(sess.run(el))

1
2
3
4
5
6
7
8
9


# methods

### flat_map vs. map

In [1]:
import tensorflow as tf

In [2]:
tf.enable_eager_execution()

In [2]:
from tensorflow.data import Dataset

In [3]:
dataset = Dataset.from_tensor_slices([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
dataset = dataset.flat_map(lambda x: Dataset.from_tensor_slices(x))
iterator = dataset.make_one_shot_iterator()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [4]:
for item in iterator:
    print(item)

TypeError: 'Iterator' object is not iterable

In [15]:
dataset = Dataset.from_tensor_slices([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
dataset = dataset.map(lambda x: Dataset.from_tensor_slices(x))
iterator = dataset.make_one_shot_iterator()

In [16]:
for item in iterator:
    print(item)
    for _item in item.take(3):
        print(_item)

<_VariantDataset shapes: (), types: tf.int32>
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
<_VariantDataset shapes: (), types: tf.int32>
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
<_VariantDataset shapes: (), types: tf.int32>
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


### dataset from generator

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.data import Dataset

In [2]:
# from generator
sequence = np.array([[[1]],[[2],[3]],[[3],[4],[5]]])
def generator():
    for el in sequence:
        yield el
dataset = Dataset.from_generator(generator,
                                           output_types= tf.int64, 
                                           output_shapes=(tf.TensorShape([None, 1]))).batch(1)
iter = dataset.make_initializable_iterator()
el = iter.get_next()
with tf.Session() as sess:
    sess.run(iter.initializer)
    print(sess.run(el))
    print(sess.run(el))
    print(sess.run(el))

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
[[[1]]]
[[[2]
  [3]]]
[[[3]
  [4]
  [5]]]


In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.data import Dataset

In [3]:
# from generator
sequence = np.array([[[1]],[[2],[3]],[[3],[4],[5]]])
def generator():
    for el in sequence:
        yield el
dataset = Dataset.from_generator(generator,
                                           output_types= tf.int64, 
                                           output_shapes=(tf.TensorShape([None, 1]))).batch(1)
iter = dataset.make_one_shot_iterator()
el = iter.get_next()
with tf.Session() as sess:
    # sess.run(iter.initializer)
    print(sess.run(el))
    print(sess.run(el))
    print(sess.run(el))

[[[1]]]
[[[2]
  [3]]]
[[[3]
  [4]
  [5]]]
